## Group 05: Abstract Cars4You

The Cars4You project aims to accelerate and standardize used-car price evaluations by replacing manual, subjective pricing with a production-ready machine learning pipeline. Our objective was to maximize predictive accuracy (MAE) on unseen cars while ensuring robustness to wrong inputs and a leakage-free evaluation.

Our EDA containing univariate, bivariate and multivariate analysis showed three dominant challenges: (1) inconsistent raw entries (typos, invalid ranges, sparse categories), (2) strong segmentation effects by brand/model, and (3) heavy-tailed numeric distributions (notably mileage). 

We addressed these with a custom engineered and reproducible sklearn pipeline. It follows the state of the art pipeline archtecture and consists the following transformers: 

deterministic cleaning and category canonicalization with `CarDataCleaner`, leakage-safe outlier handling via winsorization with `OutlierHandler`, and hierarchical, segment-aware imputation with `GroupImputer`. We then added domain-informed feature engineering with `CarFeatureEngineer` to encode depreciation, usage intensity, efficiency/performance ratios, interaction effects, and relative positioning within brand/model segments.

Encoding and scaling were consolidated in a `ColumnTransformer` combining selective log transforms, `RobustScaler`, one-hot encoding, and median target encoding for high-signal categorical structure.

To reduce noise and improve generalization, we implemented automated feature selection as a dedicated pipeline stage: VarianceThreshold followed by majority voting across complementary selectors (Spearman relevance+redundancy, mutual information, and tree-based importance). SHAP was used strictly for interpretability and diagnostics in the end.

All model selection and tuning followed a consistent 5-fold cross-validation protocol. Primary evaluation metric MAE was set at the beginning of the project, we also evaluated RMSE and R2. After a first run of different models on original and engineered features, further hyperparameter tuning on the tree-based models HistGradientBoost and RandomForest was decided. The final tuned RF pipeline improved substantially over a naive median baseline (MAE ≈ 6.8k), achieving approximately **£1.3k MAE** in cross-validation.

For detailed methodological reasoning, trade-offs and further findings please refer to the respective sections.

_exactly 300 words_

**Our approach:**
- We `clean data inconsistencies` and data entry errors that we found in the EDA
- These columns will be `set to NaN` for that specific entry to not lose rows in the data due to removing
- Afterwards, this value will be imputed (see Section 2.3)
- Also, we do category canonicalization via explicit mappings (typos/variants → one label)
- Our conservative fuzzy fallback to the mappings:
     - only applied to values that are *still missing* after deterministic mapping
     - vocabulary is learned in `fit()` from the **training fold only** → leakage-safe in CV
     - safety guardrails: minimum token length (defaults to 2) + strict similarity cutoffs

| **Feature** | **Allowed thresholds** | **Reasoning** | **# filtered below threshold** | **# filtered above threshold** |
| :--- | :--- | :--- | :---: | :---: |
| `year` | 1886 to 2020 (inclusive) | The first car was built in 1886; the dataset is from 2020, so newer cars are logically impossible. | . | ... |
| `mileage` | ≥ 0 | Negative mileage is not possible. | . | . |
| `tax` | ≥ 0 | Negative tax is not possible. | . | . |
| `mpg` | 5 to 60 (inclusive) | Realistic range for mass-market road cars. U.S. EPA list (2025): least efficient Bugatti Mistral (9 mpg), most efficient Toyota Prius (57 mpg). | . | . |
| `engineSize` | 0.6 to 12.7 (inclusive) | Practical bounds: smallest mass-production cars are in the Japanese kei class (~0.66L); very large historical production engines up to Bugatti Type 41 Royale (12.7L). | . | . |
| `paintQuality%` → `paintQuality` | 0 to 100 (inclusive) | Percentage values must be between 0 and 100. | . | . |
| `previousOwners` | ≥ 0 | Negative owner counts are not possible. | . | . |
| `hasDamage` | - | Only 0 and NaN values in the data -> no thresholding | . | . |
| TODO Categoricals | | | | |

**Legend**
- `-` means “not applicable”.